# Chaining Functions

Running one prompt at a time can produce fantastic results! Sometimes you need several outputs from your AI copilot though. You can call different plugin functions in a sequence known as a prompt chain.

The simplest version of a prompt chain would be implemented by calling InvokeAsync for each function in the order you wanted them to execute.

A more interesting version of chaining allows you to use the output of one function to feed the next function thereby modifying the final result. Let's take a look at one of these more complex prompt chains.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#!import ../config/SettingsHelper.cs
using Microsoft.SemanticKernel;

MySettings settings = Settings.LoadFromFile();
IKernelBuilder builder = Kernel.CreateBuilder();
if (settings.Type == "azure")
    builder.AddAzureOpenAIChatCompletion(
        settings.AzureOpenAI.ChatDeployment, "gpt-35-turbo", settings.AzureOpenAI.Endpoint, settings.AzureOpenAI.ApiKey);
else
    builder.AddOpenAIChatCompletion(
        settings.OpenAI.Model, settings.OpenAI.ApiKey, settings.OpenAI.OrgId);
Kernel kernel = builder.Build();

The InvokeAsync function allows you to provide one or more variables using a KernelArguments object and then any number of Kernel Functions in the order you want them to execute. The functions can be any combination of any supported types of functions including semantic, inline, or native functions. Run the following example.

In [3]:
using System.IO;

#!import Plugins/ChaosPlugin.cs
#!import Plugins/MadLibPlugin.cs

// 1. you can use any combination of semantic, native, or inline functions in a chain
string currentDir = Directory.GetCurrentDirectory();
string pluginDirectory = Path.Combine(currentDir, "Plugins");

KernelPlugin puzzlePlugin = kernel.ImportPluginFromPromptDirectory(
    Path.Combine(pluginDirectory, "PuzzlePlugin"));

KernelPlugin madLibPlugin = kernel.ImportPluginFromObject(new MadLibPlugin());
KernelPlugin chaosPlugin = kernel.ImportPluginFromObject(new ChaosPlugin());

There's nothing in the above code we haven't seen in other tutorials. To highlight how Semantic Kernel essentially treats all imported functions the same - whether they are semantic, inline, or native.

We load all of the plugins we'll need:
- The Puzzle Plugin has just one semantic function (FillTheBlanks) defined. It's job is to take a list of random adjectives, nouns, and verbs and use them to populate a mad lib template. We import the Puzzle Plugin into the Semantic Kernel by loading it it's location in our folder structure.
- The MadLib Plugin generates the mad lib template to be used by the Puzzle Plugin. The MadLib Plugin only has one function as well, but it's a native function. This function requires a theme for the generated mad lib as well as a list of numbers so it knows how many adjectives, nouns, and verb blanks to create in the mad lib. As a native plugin, we'll just create an instance of the MadLibPlugin class and import it into the Semantic Kernel. 
- Finally the Chaos Plugin - which is another native plugin, but it has two functions. The default behavior of the GenerateRandomNumbers function is to generate three integers between 0 and 5. We'll use these defaults because we only need three integers (adjectives, nouns, and verbs) and to keep our mad libs somewhat small.

In [ ]:
// define your desired parameters needed by the chain of functions and change things out and try again as desired.
// some variables will be populated by the native functions in the chain as well!
KernelArguments arguments = new()
{
    ["madLibTheme"] = "rock concert",
    ["count"] = 3,
    ["lowerBound"] = 0,
    ["upperBound"] = 5,
};

string? randomNumbers = await chaosPlugin["GenerateRandomNumbers"].InvokeAsync<string>(kernel, arguments);
string? randomWords = await chaosPlugin["GenerateRandomWords"].InvokeAsync<string>(kernel, arguments);
string? incompleteMadLib = await madLibPlugin["GenerateMadLib"].InvokeAsync<string>(kernel, arguments);
string? completeMadLib = await puzzlePlugin["FillTheBlanks"].InvokeAsync<string>(kernel, arguments);

string response = $"randomNumbers: {randomNumbers}\nrandomWords: {randomWords}\nincompleteMadLib: {incompleteMadLib}\ncompleteMadLib: {completeMadLib}";

Console.WriteLine(response);

There's actually quite a bit of code in this solution and it's spread across a few different files, but we'll walk through it here as best we can.

We are using the KernelArguments object to pass values into the first function as well as between functions. The only variable we are required to initialize is the madLibTheme variable - which we defaulted to "rock concert". The other variables will actually be created within the native plugin functions. Open the ChaosPlugin.cs and the MadLibPlugin.cs to see where they are reading and setting variables. Open the semantic function FillTheBlanks to see how it consumes the variables it needs too.
- *NOTE: Feel free to change the madLibTheme argument to generate a mad lib about something you think is interesting. :) You could also initialize arguments for lowerBound and upperBound parameters used by the GenerateRandomNumbers function. You can set them to higher integer values in order to force the AI to generate larger mad libs with more blanks to fill out.*

The second to last line of code is where it all comes together. The Semantic Kernel's InvokeAsync method takes the KernelArguments object (which just has the one madLibTheme argument initially) and then runs each of the specified functions in order. The argument are made available to all of the functions.

# Exercise

Create a new multi-function solution targeting international students learning English. You want to surprise them every day with a list of words and their definitions.

*Hint: Use the Chaos Plugin's Generate Random Numbers and Generate Random Words functions. You'll have to write your own semantic, inline, or native function to fetch the definition of words.*